In [ ]:
from flask import Flask, render_template, request, session, redirect
from flask_sqlalchemy import SQLAlchemy
import json
import os
import math
from werkzeug.utils import secure_filename
from flask_mail import Mail
from datetime import datetime

with open('config.json' , 'r') as c:
    params = json.load(c)["params"]
    
local_server = True

app = Flask(__name__)  
app.config['UPLOAD_FOLDER'] = params['upload_location']
app.secret_key = "This_is_super_Secret"
app.config.update(
    MAIL_SERVER = "smtp.gmail.com",
    MAIL_PORT = '465',
    MAIL_USE_SSL = True,
    MAIL_USERNAME = params['gmail-user'],
    MAIL_PASSWORD = params['gmail-pass']
)

mail = Mail(app)

if (local_server):
    app.config['SQLALCHEMY_DATABASE_URI'] = params['local_uri']
else:
    app.config['SQLALCHEMY_DATABASE_URI'] = params['prod_uri']

    
app.config['QLALCHEMY_TRACK_MODIFICATIONS'] = True
db = SQLAlchemy(app)

class Contact(db.Model):
    '''
    sno, name phone_num, msg, date, email
    '''
    sno = db.Column(db.Integer, primary_key=True)
    Name = db.Column(db.String(80), nullable=False)
    Ph_number = db.Column(db.String(12), nullable=False)
    Message = db.Column(db.String(120), nullable=False)
    DateTime = db.Column(db.String(12), nullable=True)
    Email = db.Column(db.String(20), nullable=False)

class Blog(db.Model):
    
    Number = db.Column(db.Integer, primary_key=True)
    Title = db.Column(db.String(20), nullable=False)
    slug = db.Column(db.String(20), nullable=False)
    Content = db.Column(db.String(250), nullable=False)
    Author = db.Column(db.String(25), nullable=True)
    DateTime = db.Column(db.String(12), nullable=True)
    img_file = db.Column(db.String(20))
    
@app.route("/dashboard", methods = ['GET', 'POST'])
def dashboard():
    #if user already logged in
    if ('user' in session and session['user']== params['admin_user'] ):
            posts = Blog.query.all()
            return render_template('dashboard.html', params = params, posts = posts)
    
    #if user is trying to login from the login page then .....
    if (request.method == 'POST'):
        #redirect to admin panel
        username = request.form.get('uname')
        userpass = request.form.get('pass')
        
        #username and password matches, render the dashboard.html template.
        if username == params['admin_user'] and userpass == params['admin_pass']:
                #set the session variable
                session['user'] = username
                posts = Blog.query.all()
                return render_template('dashboard.html', params = params, posts = posts)
    
    #in case anything fails or password and username is not typed, keep the user on the login page only, and not let the user enter to dashboard
    return render_template('login.html', params = params)

@app.route("/edit/<string:Number>", methods = ['GET', 'POST'])
def edit(Number):
        if ('user' in session and session['user']== params['admin_user'] ):
            if request.method == 'POST':
                req_title = request.form.get('Title')
                req_slug = request.form.get('Slug')
                req_content = request.form.get('Content')
                req_author = request.form.get('Author')
                req_image = request.form.get('Img_file')
                DateTime = datetime.now()
                
                if Number == '0':
                    post = Blog(Title = req_title , slug = req_slug , DateTime = DateTime, Content = req_content , Author = req_author , img_file = req_image)
                    db.session.add(post) 
                    db.session.commit()
                else:
                    post = Blog.query.filter_by(Number = Number).first()
                    post.Title = req_title
                    post.slug = req_slug
                    post.Content = req_content
                    post.Author = req_author
                    post.img_file = req_image
                    post.DateTime = DateTime
                    db.session.commit()
                    
                    return redirect("/edit/"+ Number)
            post = Blog.query.filter_by(Number = Number).first()
                
            return render_template('edit.html', params = params, Number = Number, post=post)
        
        
@app.route("/")
def home():
    
    posts = Blog.query.filter_by().all()
    last = math.ceil(len(posts)/int(params['max_posts_home']))
    
    page = request.args.get('page')
    
    if (not str(page).isnumeric()):
        page = 1
    
    page = int(page)
        
    posts = posts[(page-1)*int(params["max_posts_home"] ): (page-1)*int(params["max_posts_home"]) + int(params["max_posts_home"])]
    
    if (page == 1):
        prev = "#"
        nextpg = "/?page=" + str(page+1)
        
    elif (page == last):
        nextpg = "#"
        prev = "/?page=" + str(page-1)
    
    else:
        prev = "/?page=" + str(page-1)
        nextpg = "/?page=" + str(page+1)
        
        
    return render_template('index.html', params = params , myposts = posts, prev = prev , nextpg=nextpg)

@app.route("/about")
def about():
    return render_template('about.html' , params = params)

@app.route("/post/<string:first_post>", methods=['GET'])
def post_route(first_post):
    post = Blog.query.filter_by(slug = first_post).first()
    return render_template('post.html', params=params , post=post)

@app.route("/contact", methods = ['GET', 'POST'])
def contact():
    if(request.method=='POST'):
        '''Add entry to the database'''
        name = request.form.get('name')
        email = request.form.get('email')
        phone = request.form.get('phone')
        message = request.form.get('message')
        entry = Contact(Name=name, Ph_number = phone, Message = message, DateTime= datetime.now(),Email = email)
        db.session.add(entry)
        db.session.commit()
        mail.send_message("New Message from"+ name, sender = email ,
                          recipients=[params["gmail-user"]],
                          body = message + "\n" + phone
                         )
    return render_template('contact.html', params = params)

@app.route('/uploader' , methods = ['GET', 'POST'])
def uploader():
    if ('user' in session and session['user']== params['admin_user'] ):
            if request.method == 'POST':
                f = request.files['file1']
                f.save(os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(f.filename)))   
                return "uploaded successfully"
            
@app.route("/logout")
def logout():
    session.pop('user')
    return redirect("/dashboard")

@app.route('/delete/<string:Number>', methods=['GET','POST'])
def delete(Number):
    if ('user' in session and session['user']== params['admin_user'] ):
        post = Blog.query.filter_by(Number = Number).first()
        db.session.delete(post)
        db.session.commit()
        return redirect("/dashboard")

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


C:\Users\2016m\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Nov/2021 09:34:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2021 09:34:57] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2021 09:35:00] "GET /logout HTTP/1.1" 302 -
127.0.0.1 - - [25/Nov/2021 09:35:00] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2021 09:35:00] "GET /static/css/signin.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Nov/2021 09:35:00] "GET /docs/5.1/assets/brand/bootstrap-logo.svg HTTP/1.1" 404 -
127.0.0.1 - - [25/Nov/2021 09:35:08] "POST /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2021 09:35:18] "POST /uploader HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2021 09:35:22] "GET /dashboard HTTP/1.1"